In [1]:
import json

import pandas as pd
from db import *

/home/jovyan/work/db.py:8: SAWarning: Did not recognize type 'geometry' of column 'geom'
  mapping_hotel = Table(


In [5]:
reports_room_inventory = Table("reports_room_inventory", metadata, autoload_with=engine)

In [78]:
from datetime import date, timedelta

# Define the start and end dates
start_date = date(2023, 11, 1)
end_date = date(2024, 10, 31)

# Create a list of dates between the start and end dates
date_list = [
    start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)
]

# Create a DataFrame with the date column
df = pd.DataFrame({"date": date_list})
df["room_id"] = 3384
df["quantity"] = 0
df["basis"] = "on_request"
df["charter"] = False
df["allocation_group_id"] = 1
df["top_up"] = 0
df["stop_sale_protected"] = False
df["protected_rooms"] = 0

# Print the first few rows of the DataFrame
df.head(10)

,date,room_id,quantity,basis,charter,allocation_group_id,top_up,stop_sale_protected,protected_rooms
0,2023-11-01,3384,0,on_request,False,1,0,False,0
1,2023-11-02,3384,0,on_request,False,1,0,False,0
2,2023-11-03,3384,0,on_request,False,1,0,False,0
3,2023-11-04,3384,0,on_request,False,1,0,False,0
4,2023-11-05,3384,0,on_request,False,1,0,False,0
5,2023-11-06,3384,0,on_request,False,1,0,False,0
6,2023-11-07,3384,0,on_request,False,1,0,False,0
7,2023-11-08,3384,0,on_request,False,1,0,False,0
8,2023-11-09,3384,0,on_request,False,1,0,False,0
9,2023-11-10,3384,0,on_request,False,1,0,False,0


In [79]:
from sqlalchemy.dialects.postgresql import insert

stmt = insert(reports_room_inventory).values(df.to_dict(orient="records"))

with Session(engine) as session:
    session.execute(stmt)
    session.commit()

In [22]:
from sqlalchemy import update

stmt = (
    update(reports_room_inventory)
    .where(reports_room_inventory.c.room_id == 3183)
    .values(quantity=0)
)

with Session(engine) as session:
    session.execute(stmt)
    session.commit()

In [121]:
fanar_topup = pd.read_excel("fanar_topup.xlsx")
fanar_topup.head()

,date,topup
0,2023-12-07,5
1,2023-12-08,5
2,2023-12-14,5
3,2023-12-15,5
4,2023-12-17,5


In [61]:
for row in fanar_topup.to_dict(orient="records"):
    stmt = (
        update(reports_room_inventory)
        .where(
            reports_room_inventory.c.room_id == 3180,
            reports_room_inventory.c.date == row["date"],
            reports_room_inventory.c.allocation_group_id == 2,
        )
        .values(top_up=row["topup"])
    )

    with Session(engine) as session:
        session.execute(stmt)
        session.commit()

In [112]:
salalah_topup = pd.read_excel("salalah_topup.xlsx")
salalah_topup.head()

,date,topup
0,2023-12-07,15
1,2023-12-08,15
2,2023-12-14,15
3,2023-12-15,15
4,2024-01-21,15


In [81]:
for row in salalah_topup.to_dict(orient="records"):
    stmt = (
        update(reports_room_inventory)
        .where(
            reports_room_inventory.c.room_id == 3180,
            reports_room_inventory.c.date == row["date"],
            reports_room_inventory.c.allocation_group_id == 2,
        )
        .values(top_up=row["topup"])
    )

    with Session(engine) as session:
        session.execute(stmt)
        session.commit()

In [122]:
# add rooms to existing top up
for row in fanar_topup.to_dict(orient="records"):
    select_room = select(reports_room_inventory.c.top_up).where(
        reports_room_inventory.c.room_id == 3180,
        reports_room_inventory.c.date == row["date"],
        reports_room_inventory.c.allocation_group_id == 2,
    )

    with Session(engine) as session:
        result = session.scalar(select_room)
        
        stmt = (
            update(reports_room_inventory)
            .where(
                reports_room_inventory.c.room_id == 3180,
                reports_room_inventory.c.date == row["date"],
                reports_room_inventory.c.allocation_group_id == 2,
            )
            .values(top_up=row["topup"] + result)
        )
        session.execute(stmt)
        session.commit()
        new_result = session.scalar(select_room)

        print(row["topup"], result, new_result)

5 15 20
5 16 21
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 7 12
5 7 12
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 8 13
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
5 0 5
